In [64]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import requests 
from bs4 import BeautifulSoup
import time
import math
import pandas as pd
from google.colab import drive
drive.mount('drive')

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:14 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 Packages [2,167 kB]
Get:15 http://

In [65]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
while True:
  driver.get('https://www.google.com/search?q=us+election+results&oq=us+election+results&aqs=chrome.0.69i59j0i131i433l6j69i60.8640j0j9&sourceid=chrome&ie=UTF-8')
  timeout = 3
  try:
      element_present = EC.presence_of_element_located((By.ID, '_83GlX73aK5_G4-EPqtC9uAQ95'))
      WebDriverWait(driver, timeout).until(element_present)
  except TimeoutException:
      print("Timed out waiting for page to load")
  finally:
      print("Page loaded")
      userid_element = driver.find_elements_by_xpath('//div[@Style="width:172px;text-align:center"]')[0]
      userid_element.click()
      time.sleep(1)
      userid_element = driver.find_elements_by_xpath('//span[@class="PLaf5"]')
      userid_element1 = driver.find_elements_by_xpath('//span[@class="E2WDKf"]')


      data = []
      for i in userid_element:
        if i.text != "":
          data.append(i.text)
      for i in userid_element1:
        if i.text != "":
          data.append(int(i.text.replace('% reporting','')))
      for i in range(int(len(data)/2)):
        data.append(int(driver.find_elements_by_xpath('//div[@class="oLhH0e"]')[i].find_elements_by_tag_name('span')[1].text.replace(',','')))
        data.append(int(driver.find_elements_by_xpath('//div[@class="oLhH0e"]')[i].find_elements_by_tag_name('span')[3].text.replace(',','')))



      print(data)
      rawData = []
      for i in range(int(len(data)/4)):
        d = []
        for j in range(2):
          d.append(data[i+j*6])
        d.append(data[12+i*2])
        d.append(data[13+i*2])
        rawData.append(d)
      print(rawData)

      
      info =[]
      data = rawData
      for i in range(len(data)):
        raw = data[i]
        d =[]
        d.append(raw[0])
        diff = raw[2] - raw[3]
        percent = raw[1]/100.0
        uncountedVotes = math.floor((raw[2]+raw[3])*(1-percent)/percent)
        if diff < 0:
          voteNeeded = math.ceil(((uncountedVotes + diff)/2) -diff)
        else:
          voteNeeded = math.ceil(((uncountedVotes - diff)/2))

        percentNeeded = voteNeeded/(uncountedVotes*1.0)
        d.append(str(raw[1])+"%")
        d.append(raw[2])
        d.append(raw[3])
        if percentNeeded < 1 and voteNeeded > 0:
          d.append(uncountedVotes)
          d.append(voteNeeded)
          d.append(math.ceil(percentNeeded*1000)/1000.0)
          d.append(uncountedVotes+1-voteNeeded)
          d.append(math.ceil((((uncountedVotes+1-voteNeeded)/uncountedVotes)*1000))/1000.0)
          info.append(d)
          print(d)
        elif voteNeeded <= 0:
          d.append(uncountedVotes)
          d.append('Already Won')
          d.append('Already Won')
          d.append('Impossible to Win in This World')
          d.append('Impossible to Win in This World')
          info.append(d)
          print(d)
        elif percentNeeded >= 1:
          d.append(uncountedVotes)
          d.append('Impossible to Win in This World')
          d.append('Impossible to Win in This World')
          d.append('Already Won')
          d.append('Already Won')
          info.append(d)
          print(d)

      print(info)
      df = pd.DataFrame (info,columns=['State Name','Percentage of Counted Votes','Counted Votes for Biden','Counted Votes for Trump','Uncounted Votes','Vote Biden Needed to Win','Percentage of Uncounted Vote Biden Needed to Win','Vote Trump Needed to Win','Percentage of Uncounted Vote Biden Needed to Win'])
      mainPath =r'drive/My Drive/New Folder'
      csvPath =mainPath+"/"+"USElectionSwingState.csv"
      df.to_csv(csvPath)
      time.sleep(120)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


Timed out waiting for page to load
Page loaded
['Georgia', 'Nevada', 'North Carolina', 'Pennsylvania', 'Arizona', 'Florida', 99, 87, 99, 98, 90, 99, 2450385, 2448799, 627104, 606967, 2655383, 2732084, 3311780, 3298107, 1562200, 1521246, 5284377, 5658690]
[['Georgia', 99, 2450385, 2448799], ['Nevada', 87, 627104, 606967], ['North Carolina', 99, 2655383, 2732084], ['Pennsylvania', 98, 3311780, 3298107], ['Arizona', 90, 1562200, 1521246], ['Florida', 99, 5284377, 5658690]]
['Georgia', '99%', 2450385, 2448799, 49486, 23950, 0.484, 25537, 0.517]
['Nevada', '87%', 627104, 606967, 184401, 82132, 0.446, 102270, 0.555]
['North Carolina', '99%', 2655383, 2732084, 54418, 'Impossible to Win in This World', 'Impossible to Win in This World', 'Already Won', 'Already Won']
['Pennsylvania', '98%', 3311780, 3298107, 134895, 60611, 0.45, 74285, 0.551]
['Arizona', '90%', 1562200, 1521246, 342605, 150826, 0.441, 191780, 0.56]
['Florida', '99%', 5284377, 5658690, 110536, 'Impossible to Win in This World', 

ElementClickInterceptedException: ignored